## Created by <a href="https://github.com/yunsuxiaozi/">yunsuxiaozi</a>  2024/9/29

### 这是<a href="https://www.datafountain.cn/competitions/1045">基于运营商文本数据的知识库检索</a>比赛的baseline.这里将使用Kaggle平台和<a href="https://github.com/gomate-community/GoMate">GoMate</a>框架来完成。

### GoMate框架做RAG任务可以参考<a href="https://www.kaggle.com/code/yunsuxiaozi/gomate-framework-usage">Gomate framework usage</a>。



# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">1.将PDF文件解析为txt文件</h1></span>

In [1]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.7 MB/s eta 0:00:00


In [2]:
import PyPDF2
import os
from tqdm import tqdm
document_path="/kaggle/input/2024ccf-retrieval/retrieval/A_document/"
pdfs=sorted(os.listdir(document_path))
texts=''
for i in tqdm(range(len(pdfs))):
    with open(f'{document_path}{pdfs[i]}', 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        # 读取每一页
        for page_num in range(len(reader.pages)):
            # 提取文本
            page_obj = reader.pages[page_num]
            text = page_obj.extract_text()
            texts+=text
    texts+='\n'
print(f"len(texts):{len(texts)}")
final_texts=''
def is_chinese_char(uchar):
    if '\u4e00' <= uchar <= '\u9fff' or \
       '\u3400' <= uchar <= '\u4dbf' or \
       '\u20000' <= uchar <= '\u2a6df' or \
       '\u2a700' <= uchar <= '\u2b73f' or \
       '\u2b740' <= uchar <= '\u2b81f' or \
       '\u2b820' <= uchar <= '\u2ceaf' or \
       '\u2ceb0' <= uchar <= '\u2ebef' or \
       '\u30000' <= uchar <= '\u3134f':
        return True
    else:
        return False
for i in range(len(texts)):
    if is_chinese_char(texts[i]):
        final_texts+=texts[i]

with open('corpus.txt', 'w', encoding='utf-8') as f:
    f.write(final_texts)

100%|██████████| 120/120 [00:59<00:00,  2.02it/s]


len(texts):847111


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">2.GoMate框架的配置</h1></span>

In [3]:
!git clone https://github.com/gomate-community/GoMate.git #gomate是配置化模块化的RAG框架,可靠的输入和可信的输出

Cloning into 'GoMate'...
remote: Enumerating objects: 1374, done.
remote: Counting objects: 100% (501/501), done.
remote: Compressing objects: 100% (237/237), done.
remote: Total 1374 (delta 312), reused 428 (delta 259), pack-reused 873 (from 1)
Receiving objects: 100% (1374/1374), 55.89 MiB | 28.60 MiB/s, done.
Resolving deltas: 100% (799/799), done.


In [4]:
cd /kaggle/working/GoMate

/kaggle/working/GoMate


In [5]:
!pip install -q -e .  #安装依赖

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.6.0 which is incompatible.
distributed 2024.7.1 requires dask==2024.7.1, but you have dask 2024.9.0 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1 which is incompatible.
jupyterlab 4.2.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which 

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">3.文本的分块</h1></span>

In [6]:
from tqdm import tqdm#进度条库   tqdm(range(10))
from gomate.modules.document.chunk import TextChunker#对文本进行分块的类
from gomate.modules.document.txt_parser import TextParser#文本解析器
#将文本解析为句子的列表
def generate_chunks(path):
    #文本解析器和文本分块器
    tp = TextParser()
    tc = TextChunker()
    #从txt文件里解析文本,分成段落
    paragraphs = tp.parse(path, encoding="utf-8")
    #段落拆分成句子
    chunks = []
    for content in tqdm(paragraphs):
        chunk = tc.chunk_sentences([content], chunk_size=64)
        chunks.append(chunk)
    return chunks
chunks=generate_chunks(path='/kaggle/working/corpus.txt')
#这里将[['a'],['b'],['c']]变成['a','b','c'],这是我们的语料库
corpus = []
for chunk in chunks:
    corpus.extend(chunk)

RAG Vocab: /kaggle/working/GoMate/gomate/modules/document/huqie.txt
RAG Trie: /kaggle/working/GoMate/gomate/modules/document/huqie.txt.trie


100%|██████████| 1/1 [02:10<00:00, 130.49s/it]


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">4.检索器设置</h1></span>

In [7]:
from gomate.modules.retrieval.bm25s_retriever import BM25RetrieverConfig#基于BM25的检索器
from gomate.modules.retrieval.dense_retriever import DenseRetrieverConfig#基于稠密向量表示的检索器
from gomate.modules.retrieval.hybrid_retriever import HybridRetriever, HybridRetrieverConfig#实现混合检索器的类


bm25_config = BM25RetrieverConfig(
    method='lucene',
    index_path='indexs/description_bm25.index',
    k1=1.6,
    b=0.7
)
bm25_config.validate()

dense_config = DenseRetrieverConfig(
    model_name_or_path="BAAI/bge-large-zh-v1.5",
    dim=1024,
    index_path='indexs/dense_cache'
)
config_info = dense_config.log_config()


# 由于分数框架不在同一维度，建议可以合并
hybrid_config = HybridRetrieverConfig(
    bm25_config=bm25_config,
    dense_config=dense_config,
    bm25_weight=0.7,  # bm25检索结果权重
    dense_weight=0.3  # dense检索结果权重
)
hybrid_retriever = HybridRetriever(config=hybrid_config)
#构建索引
hybrid_retriever.build_from_texts(corpus)
#保存索引
hybrid_retriever.save_index()
# 加载索引
hybrid_retriever.load_index()

BM25 configuration is valid.


tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

BM25S Count Tokens:   0%|          | 0/6549 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/6549 [00:00<?, ?it/s]

Building index: 100%|██████████| 205/205 [02:27<00:00,  1.39it/s]


Finding newlines for mmindex:   0%|          | 0.00/2.08M [00:00<?, ?B/s]

Index saving to：indexs/dense_cache
Index saved successfully to indexs/dense_cache
Index loaded successfully from indexs/dense_cache


# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">5.问题的检索和重排</h1></span>

In [8]:
import pandas as pd
submission=pd.read_csv("/kaggle/input/2024ccf-retrieval/retrieval/submit_example.csv")


from gomate.modules.reranker.bge_reranker import BgeRerankerConfig, BgeReranker#基于bert的重排器
reranker_config = BgeRerankerConfig(
    model_name_or_path='BAAI/bge-reranker-large'
)
bge_reranker = BgeReranker(reranker_config)

question=list(submission['question'].values)

answer=[]
topk=5
for q in question:

    search_docs = hybrid_retriever.retrieve(q,top_k=topk)
    search_docs = bge_reranker.rerank(
        query=q,
        documents=[doc['text'] for idx, doc in enumerate(search_docs)]
    )
    #截断防止超长度
    answer.append( ''.join([doc['text'] for idx, doc in enumerate(search_docs)]) )
print(f"answer[0]:{answer[0]}")
for i in range(len(answer)):
    print(len(answer[i]))
answer=[a[:500] for a in answer]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

...

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

answer[0]:我们坚定践行网络强国、数字中国、智慧社会战略部署今天的中国联通正在从传统运营商加速向数字科技领军企业转变实现了四个维度的转型升级一是联接规模和联接结构升维从过去的连接人为主拓展到连接人机物大力发展物联网和工业互联网二是核心功能升维从以基础连接为主发展到大联接、大计算、大数据、大应用、大安全五大主责主业三是服务和赋能水平升维以、云计算、大数据、人工智能、区块链为代表的新一代信息技术和实体经济的结合服务数字政府、数字社会、数字经济的能力不断增强四是发展理念升维我们以传统的市场驱动为主转变为市场驱动和创新驱动相结合的发展模式尤其是加大了科技创新及人才方面的投入力度创新发展的动能得到了空前的释放。公司正在加快实现从数量型规模型向质量型效益效率型转变为科技创新转型奠定坚实基础。企业转型从数字化走向智能化让计算和网络融合的算力网络成为刚需。年中国联通发布新型数字基础设施计划提出了全面推动网络从提供“尽力而为”的连接向提供内生确定性服务转变的理念。在这过程中中国联通不仅要发挥作为“国家队”的责任担当还要追求作为“排头兵”的自我超越。具体而言中国联通将通过通信科技、信息科技、数字科技和运营科技等领域的融合创新推动企业从传统的通信服务提供商向综合性的科技服务企业转型升级。今天的中国联通正在加速转型为数字科技领军企业我们伴随中国特色估值体系重塑提振的东风扬帆起航积极为“中国特色现代资本市场”高质量发展添砖加瓦。公司积极响应国资委的号召以全面深化国企改革为动力坚定不移地做强做优做大国有资本和国有企业。在改革过程中中国联通注重创新引领以科技创新为核心驱动力推动企业转型升级和高质量发展。综上所述中国联通在技术创新和智能化生态的打造方面取得了显著成效。未来随着数字经济的不断深入发展中国联通将继续秉持创新驱动的发展理念不断提升数据管理能力和智能化水平为行业的数字化转型和智能化升级贡献更多力量。通过不断创新和突破中国联通正引领着数字化转型的新浪潮为我国乃至全球的数字化发展贡献着重要力量。未来随着倡议的深入实施和数字化转型的持续推进中国联通必将在更广阔的舞台上展现出更加耀眼的光芒。中国联合网络通信股份有限公司年年度报告摘要营销渠道年积极落实全面数字化转型战略依托中台高效支撑强化全渠道协同重点实施“‘’”工程加快推进在线线下一化构建增存一体、服务与营销一体的数字化渠道运营体系。
9

# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">6.提交文件</h1></span>

In [9]:
from transformers import AutoTokenizer, AutoModel#自动加载分词器和自动加载预训练模型
import pandas as pd#读取表格数据的库
import numpy as np
import torch#深度学习框架pytorch
from tqdm import tqdm
print("load embedding tokenizer")
#允许远程加载模型,模型加载的时候将模型转成半精度浮点数并移动到GPU上。
embed_tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-large-zh-v1.5", trust_remote_code=True)
print("load embedding model")
embed_model=AutoModel.from_pretrained("BAAI/bge-large-zh-v1.5",trust_remote_code=True)
#模型转换成评估模式
embed_model.eval()
def embedding(texts,batch_size=5):
    embed_vector=[]
    for i in tqdm(range(0,len(texts),batch_size)):
        text_input=embed_tokenizer(texts[i:i+batch_size], max_length=1024, padding=True, truncation=True,
                                               return_tensors='pt')
        with torch.no_grad():
            model_output = embed_model(**text_input)
        #,截取开头的cls,据说cls有了全局的特征,所以[:, 0]就是cls pool.
        batch_embeddings=model_output['last_hidden_state'][:, 0]
        #(batch_size,1024)
        batch_embeddings=torch.nn.functional.normalize(batch_embeddings, p=2, dim=1)
        embed_vector.append(batch_embeddings.numpy())
    return np.vstack(embed_vector)

load embedding tokenizer
load embedding model


In [10]:
submission['answer']=answer
answer_vector=embedding(answer)
answer_embedding=[ ','.join([str(a) for a in answer_vector[i]]) for i in range(len(answer_vector))]
submission['embedding']=answer_embedding
submission.to_csv("gomate_baseline.csv",index=None)
submission.head()

100%|██████████| 20/20 [02:32<00:00,  7.63s/it]


,ques_id,question,answer,embedding
0,1,根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？,我们坚定践行网络强国、数字中国、智慧社会战略部署今天的中国联通正在从传统运营商加速向数字科技...,"-0.02454025,-0.008587598,-0.033859223,-0.00594..."
1,2,告诉我2022年联通产业互联网收入的同比增长速度。,年公司创新业务保持快速发展产业互联网收入同比增长达到人民币亿元占整体主营业务收入比例达到成为...,"-0.02003257,-0.02254348,-0.027921254,-0.006499..."
2,3,根据2022年度报告，中国联通的企业定位是什么？,中国联通将继续围绕国家所需、产业链供应链所困以战略性新兴产业和未来产业为主线全力成为国家战略...,"-0.039004676,-0.03214621,-0.042470366,-0.01483..."
3,4,2022年联通在“大联接”和“大数据”业务上取得了什么成果？,“大联接”方面中国联通抢抓“双千兆”“物超人”发展机遇坚持量质构效协同发展。联通云加速发展实...,"-0.015529217,-0.006176334,-0.033465676,-0.0054..."
4,5,2022年上半年，联通在精品网络建设上有什么成果？,适度加大战略投入基础网络能力大幅提升中国联通始终坚持网络在企业发展中的基础地位适度加大战略投...,"-0.0028932593,-0.0139340125,-0.039516766,-0.00..."
